In [2]:
import numpy as np
import os

In [3]:
pathway = '/Volumes/Jiamu2/Pypeit1.3.0/2020feb26_pypeit_red/hgc1a_results/1.3.2_test/keck_deimos_A/det37_new'
possible_paths = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(pathway)) for f in fn]

In [4]:
spec1d_files = []
spec1d_dict = {}
for file in possible_paths:
    if ('spec1d_' in file) and ('.txt' in file):
        spec1d_files.append(file)
        last_slash_ind = file.rfind('/')
        path = file[:last_slash_ind+1]
        if path not in spec1d_dict:
            spec1d_dict[path] = []
        spec1d_dict[path].append(file[last_slash_ind+1:])

In [5]:
spec1d_files

['/Volumes/Jiamu2/Pypeit1.3.0/2020feb26_pypeit_red/hgc1a_results/1.3.2_test/keck_deimos_A/det37_new/Science/spec1d_d0226_0061-hgc1a_DEIMOS_2020Feb26T055522.915.txt',
 '/Volumes/Jiamu2/Pypeit1.3.0/2020feb26_pypeit_red/hgc1a_results/1.3.2_test/keck_deimos_A/det37_new/Science/spec1d_d0226_0062-hgc1a_DEIMOS_2020Feb26T061443.181.txt',
 '/Volumes/Jiamu2/Pypeit1.3.0/2020feb26_pypeit_red/hgc1a_results/1.3.2_test/keck_deimos_A/det37_new/Science/spec1d_d0226_0063-hgc1a_DEIMOS_2020Feb26T063353.251.txt']

In [11]:
tol_dict = {'slit':20,'spat_pixpos':20,'s2n':100} 
snr_thresh = 0.1

for path in spec1d_dict: 
    spec1d_texts = spec1d_dict[path]
    spec1d_contents = {}
    for file in spec1d_texts: #loop over the different observations of the same mask on the same date
        spec1d_contents[file] = {}
        with open(f'{path}{file}','r') as f: #read each spec1d.txt file
            lines = f.readlines()
        labels = lines[0].strip().split('|') #get the labels
        for j in range(len(labels)):
            labels[j] = labels[j].strip()
        while '' in labels:
            labels.remove('')
        labels.append('det_num')
        data = []
        for line in lines[1:]: #get the data
            line = line.strip().split('|')
            while '' in line:
                line.remove('')
            for j in range(len(line)):
                line[j] = line[j].strip()
                if '-DET' in line[j]:
                    det_num = line[j].split('-')[-1]
                try:
                    line[j] = float(line[j])
                except:
                    pass
            line.append(det_num)
            data.append(line)
        #save the labels and data into a dictionary
        for j in range(len(labels)):
            if labels[j] not in spec1d_contents[file]:
                spec1d_contents[file][labels[j]] = []
            for row in data:
                spec1d_contents[file][labels[j]].append(row[j])
            spec1d_contents[file][labels[j]] = np.array(spec1d_contents[file][labels[j]])
    internal_matches = {} #find matches within each spec1d text file
    filenames = spec1d_contents.keys()
    print(spec1d_contents[file]['objname'])
    for j,file in enumerate(filenames):
        first_det = -1
        internal_matches[file] = []
        for k in range(len(spec1d_contents[file]['name'])):
            if first_det == -1:
                first_det = spec1d_contents[file]['det_num'][k]
            if spec1d_contents[file]['s2n'][k] < snr_thresh:
                continue
            match_params = []
            current_matches = np.array(internal_matches[file])

            for kk in range(len(spec1d_contents[file]['name'])):
#                 if len(current_matches) > 0:
#                     if (spec1d_contents[file]['det_num'][kk] != first_det) and \
#                        (spec1d_contents[file]['name'] in current_matches[:,1]):
#                         continue
                if k == kk:
                    continue
                elif spec1d_contents[file]['s2n'][kk] < snr_thresh:
                    continue
                elif spec1d_contents[file]['det_num'][k] == spec1d_contents[file]['det_num'][kk]:
                    continue
                elif spec1d_contents[file]['objname'][k] != spec1d_contents[file]['objname'][kk]:
                    continue
                else:
                    match_params.append((k,kk,
                                         np.abs(spec1d_contents[file]['slit'][k]-spec1d_contents[file]['slit'][kk]),
                                         np.abs(spec1d_contents[file]['spat_pixpos'][k]-spec1d_contents[file]['spat_pixpos'][kk]),
                                         np.abs(spec1d_contents[file]['s2n'][k]-spec1d_contents[file]['s2n'][kk]),
                                        ))
            match_params = np.array(match_params)
            if len(match_params) == 0:
                continue
            elif np.min(match_params[:,2]) > tol_dict['slit']:
                continue
            good_matches = (match_params[:,2] <= tol_dict['slit']) &\
                           (match_params[:,3] <= tol_dict['spat_pixpos']) &\
                           (match_params[:,4] <= tol_dict['s2n'])
            if np.sum(good_matches) == 0:
                continue
            elif np.sum(good_matches) > 1: ####NEED TO WRITE THIS SECTION!
                slit_diffs = match_params[good_matches][:,2]
                spat_diffs = match_params[good_matches][:,3]
                s2n_diffs = match_params[good_matches][:,4]
                if np.all(slit_diffs == slit_diffs[0]): #then they share slit numbers
                    if np.all(spat_diffs == spat_diffs[0]):
                        if np.all(s2n_diffs == s2n_diffs[0]):
                            print(match_params[good_matches])
                            raise ValueError('ERROR: All metrics match, so no pair was found.')
                        else:
                            best_match = np.argmin(s2n_diffs)
                    else:
                        best_match = np.argmin(spat_diffs)
                else:
                    best_match = np.argmin(slit_diffs)
                slice_vals = match_params[good_matches][best_match]
                inds = np.array([slice_vals[0],slice_vals[1]]).astype(int)
            else: #one match
                slice_vals = match_params[good_matches][0]
                inds = np.array([slice_vals[0],slice_vals[1]]).astype(int)
            det_nums = spec1d_contents[file]['det_num'][inds]
            det1,det2 = int(det_nums[0][3:]),int(det_nums[1][3:])
            if det1 > det2:
                inds = np.array([inds[1],inds[0]])
            current_pair = spec1d_contents[file]['name'][inds]
            already_paired = False
            for row in current_matches:
                if (current_pair[0] == row[0]) and (current_pair[1] == row[1]):
                    already_paired = True
                    continue
            if not already_paired:
                internal_matches[file].append(current_pair)
        internal_matches[file] = np.array(internal_matches[file])
        
    external_matches = [] #find matches between spec1d text files
    for j,file in enumerate(filenames):
        first_det = -1
        for k in range(len(spec1d_contents[file]['name'])):
            if first_det == -1:
                first_det = spec1d_contents[file]['det_num'][k]
            if spec1d_contents[file]['s2n'][k] < snr_thresh:
                continue
            
            for file2 in filenames:
                match_params = []
                current_matches = np.copy(external_matches)
                if file == file2:
                    continue
                for kk in range(len(spec1d_contents[file2]['name'])):
    #                 if len(current_matches) > 0:
    #                     if (spec1d_contents[file]['det_num'][kk] != first_det) and \
    #                        (spec1d_contents[file]['name'] in current_matches[:,1]):
    #                         continue
                    if k == kk:
                        continue
                    elif spec1d_contents[file2]['s2n'][kk] < snr_thresh:
                        continue
                    elif spec1d_contents[file]['det_num'][k] != spec1d_contents[file2]['det_num'][kk]:
                        continue
                    else:
                        match_params.append((k,kk,
                                             np.abs(spec1d_contents[file]['slit'][k]-spec1d_contents[file2]['slit'][kk]),
                                             np.abs(spec1d_contents[file]['spat_pixpos'][k]-spec1d_contents[file2]['spat_pixpos'][kk]),
                                             np.abs(spec1d_contents[file]['s2n'][k]-spec1d_contents[file2]['s2n'][kk]),
                                             ))
                match_params = np.array(match_params)
                if len(match_params) == 0:
                    continue
                elif np.min(match_params[:,2]) > tol_dict['slit']:
                    continue
                good_matches = (match_params[:,2] <= tol_dict['slit']) &\
                               (match_params[:,3] <= tol_dict['spat_pixpos']) &\
                               (match_params[:,4] <= tol_dict['s2n'])
                if np.sum(good_matches) == 0:
                    continue
                elif np.sum(good_matches) > 1: ####NEED TO WRITE THIS SECTION!
                    slit_diffs = match_params[good_matches][:,2]
                    spat_diffs = match_params[good_matches][:,3]
                    s2n_diffs = match_params[good_matches][:,4]
                    if np.all(slit_diffs == slit_diffs[0]): #then they share slit numbers
                        if np.all(spat_diffs == spat_diffs[0]):
                            if np.all(s2n_diffs == s2n_diffs[0]):
                                print(match_params[good_matches])
                                raise ValueError('ERROR: All metrics match, so no pair was found.')
                            else:
                                best_match = np.argmin(s2n_diffs)
                        else:
                            best_match = np.argmin(spat_diffs)
                    else:
                        best_match = np.argmin(slit_diffs)
                    slice_vals = match_params[good_matches][best_match]
                else: #one match
                    slice_vals = match_params[good_matches][0]
                inds = np.array([slice_vals[0],slice_vals[1]]).astype(int)
                current_pair = [file+'/'+spec1d_contents[file]['name'][inds[0]],
                                file2+'/'+spec1d_contents[file2]['name'][inds[1]]]
                already_paired = False
                for row in current_matches:
                    if (current_pair[0] == row[0]) and (current_pair[1] == row[1]):
                        already_paired = True
                        continue
                    elif (current_pair[1] == row[0]) and (current_pair[0] == row[1]):
                        already_paired = True
                        continue
                if not already_paired:
                    external_matches.append(current_pair)
    external_matches = np.array(external_matches)
    final_matches = {}
    added_matches = []
    for pair in external_matches:
        entry1,entry2 = pair
        f1,f2 = entry1.split('/')[0],entry1.split('/')[1]
        spat1,spat2 = entry1.split('-SLIT')[0][-4:],entry2.split('-SLIT')[0][-4:]
        all_matches = [entry1,entry2]
        for pair2 in external_matches:
            p2_entry1,p2_entry2 = pair2
            if (entry1 == p2_entry1) and (entry2 == p2_entry2):
                continue
            elif (entry2 == p2_entry1) and (entry1 == p2_entry2):
                continue
            elif ((entry1 != p2_entry1) and (entry2 != p2_entry2)) and \
                 ((entry2 != p2_entry1) and (entry1 != p2_entry2)):
                continue
            for entry in pair2:
                if entry not in all_matches:
                    all_matches.append(entry)
        not_already_added = False
        for entry in all_matches:
            if entry not in added_matches:
                not_already_added = True
        n_current_matches = len(all_matches)
        for ind in range(n_current_matches):
            entry = all_matches[ind]
            f,name = entry.split('/')
            poss_int_matches = internal_matches[f]
            #print(poss_int_matches[0])
            for pair2 in poss_int_matches:
                if (name == pair2[0]):
                    all_matches.append(f+'/'+pair2[1])
                    break
                elif (name == pair2[1]):
                    all_matches.append(f+'/'+pair2[0])
                    break
        
        not_already_added = False
        spat_vals = []
        for entry in all_matches:
            if entry not in added_matches:
                not_already_added = True
            spat_vals.append(int(entry.split('SPAT')[1][:4]))
        ave_spat = '%04d'%int(round(np.average(spat_vals)))
        if not_already_added:
            if ave_spat in final_matches:
                print(f'\n\nConflict with SPAT{ave_spat}')
                print('Already in dict:')
                for line in final_matches[ave_spat]:
                    print(line)
                print('New proposal:')
                for line in all_matches:
                    print(line)
                if len(final_matches[ave_spat]) < len(all_matches):
                    final_matches[ave_spat] = all_matches
            else:
                final_matches[ave_spat] = all_matches
            added_matches.extend(final_matches[ave_spat])
#             if len(all_matches) not in [6]:
#                 print(all_matches,len(all_matches))
#                 for line in all_matches:
#                     print(line)
    outpath = f'{path}coadd1d_results/'
    #if not os.path.isdir(outpath):
        #os.makedirs(outpath)
    #for spat in final_matches:
        #with open(f'{outpath}SPAT{spat}_coadd1d_instruct.txt','w') as f:
            #f.write("# User-defined coadding parameters\n")
            #f.write("[coadd1d]\n")
            #f.write(f"    coaddfile = 'SPAT{spat}_coadd1d.fits'\n")
            #f.write("    flux_value = False\n\n")
            #f.write("# Read in the data\n")
            #f.write("coadd1d read\n")
            ###LOOP OVER THE FILES!
            #for file in final_matches[spat]:
                #filepath,entry = file.split('/')
                #spec1d_filepath = filepath.split('.txt')[0]+'.fits'
                #f.write(f"    ../{spec1d_filepath} {entry}\n")
            #f.write("coadd1d end")

['G_65703339124992' '780896.0' '779296.0' 'G_65741993950848'
 'G_66631052293120' 'G_66631052064768' 'G_66631052064256'
 'G_66699771952128' '789063.0' '793082.0' 'G_66596692325376'
 'G_66596693805440' '788401.0' 'G_65016144356864' '806552.0'
 'G_64844345661312' 'G_65084863834752' '789605.0' 'G_66871570238592'
 'G_66768491017856' 'G_65703339124992' '780896.0' 'G_65741993950848'
 'G_66631052293120' 'G_66631052064768' 'G_66631052064256'
 'G_66699771952128' '789063.0' '793082.0' 'G_66596692325376'
 'G_66596693805440' 'G_65016144356864' 'G_64844345661312' 'SERENDIP'
 '793902.0' 'SERENDIP' 'G_65084863834752' 'G_66871570238592'
 'G_66768491017856']
